### Action of Steenrod algebra on Stiefel-Whitneys

In [153]:
import itertools

In [1]:
F2 = GF(2)
Sym = SymmetricFunctions(F2).elementary()

In [2]:
def binomial_mod_2(n, k):
    '''Returns binomial(n, k) mod 2. Requires 0 <= k <= n.'''
    while n > 0:
        if is_even(n) and is_odd(k): return 0
        n //= 2
        k //= 2
    return 1

In [3]:
def sw_monomial(J):
    '''Returns the Stiefel-Whitney monomial w^J'''
    return Sym._from_dict({J: F2.one()})

In [4]:
def sw_polynomial(Js):
    '''Returns the Stiefel-Whitney polynomial sum(w^J for J in Js). Requires Js to consist of distinct partitions.'''
    return Sym._from_dict({J: F2.one() for J in Js})

In [5]:
@cached_function
def Sq_single_cached(i, j):
    '''Apply Sq^i to a single Stiefel-Whitney class w_j, caching the result. Requires i <= j.'''
    if i == j:
        # Use cup square
        return sw_monomial(Partition([i, i]))
    else:
        # Use Wu formula
        return sw_polynomial((Partition([j + t, i - t]) for t in (0..i) if binomial_mod_2(j + t - i - 1, t) == 1))

In [6]:
@cached_function
def Sq_monomial_cached(i, J):
    '''Apply Sq^i to a Stiefel-Whitney monomial w^J, caching the result. Requires i <= sum(J).'''

    if len(J) <= 1:
        # Single Stiefel-Whitney class
        return Sq_single_cached(i, sum(J))

    # Product of several Stiefel-Whitney classes - use Cartan formula.
    j = J[0]
    J2 = Partition(J[1:])
    return sum(Sq_single_cached(i2, j) * Sq_monomial_cached(i - i2, J2) for i2 in (max(0, i - sum(J2))..min(i, j)))

In [7]:
def Sq_monomial(i, J):
    '''Apply Sq^i to a Stiefel-Whitney monomial w^J'''

    # Deal with degree check before hitting cache.
    if i > sum(J): return Sym.zero()

    return Sq_monomial_cached(i, J)

In [8]:
def Sq(i, w):
    '''Apply Sq^i to a Stiefel-Whitney polynomial w'''

    # Handle trivial case trivially
    if i == 0: return w

    # Apply to each monomial
    return sum(Sq_monomial(i, J) for J in w.support())

The following example calculates $\text{Sq}^4(w_3 w_1) = w_3^2 w_1^2$:

In [9]:
Sq(4, sw_monomial(Partition([3, 1])))

e[3, 3, 1, 1]

### Wu classes

In [10]:
@cached_function
def wu_class(n):
    '''Returns the nth Wu class'''
    return sw_monomial(Partition([n])) + sum(Sq(i, wu_class(n - i)) for i in (1..n))

In [11]:
[wu_class(n) for n in (0..5)]

[e[],
 e[1],
 e[1, 1] + e[2],
 e[2, 1],
 e[1, 1, 1, 1] + e[2, 2] + e[3, 1] + e[4],
 e[2, 1, 1, 1] + e[2, 2, 1] + e[3, 1, 1] + e[4, 1]]

### Wu relations

In [12]:
def wu_relations(n):
    '''Returns all Wu relations among Stiefel-Whitney polynomials at degree n'''
    for m in (0..n):
        for J in Partitions(m):
            wJ = sw_monomial(J)
            rel = wu_class(n - m) * wJ + Sq(n - m, wJ)
            if rel.is_zero(): continue
            yield rel

In [13]:
[x for x in wu_relations(4)]

[e[1, 1, 1, 1] + e[2, 2] + e[3, 1] + e[4],
 e[2, 1, 1],
 e[2, 1, 1],
 e[2, 1, 1],
 e[2, 1, 1] + e[3, 1]]

In [39]:
@cached_function
def partitions_reverse_dict(n):
    '''Enumerate Partitions(n), and return a dict mapping partitions to their indexes.'''
    return {J: i for i, J in enumerate(Partitions(n))}

In [62]:
def partition_to_index(J):
    '''Return the index of J within Partitions(n(J)).'''
    return partitions_reverse_dict(sum(J))[J]

In [266]:
def relations_to_matrix(rels, n):
    '''Given a set of relations among Stiefel-Whitney numbers at degree n, represent it by a matrix with a row for each relation.'''
    max_row = -1
    def rowcol_generator():
        nonlocal max_row
        for row, rel in enumerate(rels):
            max_row = row
            for J in rel.support():
                yield (row, partition_to_index(J))
    entries = {rowcol: F2.one() for rowcol in rowcol_generator()}
    nrows = max_row + 1

    return matrix(F2, nrows, Partitions(n).cardinality(), entries, sparse=True)

In [270]:
def relations_to_subspace(rels, n):
    '''Given a set of relations among Stiefel-Whitney numbers at degree n, return the subspace of Stiefel-Whitney numbers that obey it.'''
    return relations_to_matrix(rels, n).right_kernel()

In [271]:
@cached_function
def stiefel_whitneys(n):
    '''Return all Stiefel-Whitney numbers of unoriented n-manifolds.'''
    return relations_to_subspace(wu_relations(n), n)

#### Example

The following example shows that the only nontrivial Stiefel-Whitney number of a 5-manifold is $w_2 w_3$:

In [272]:
stiefel_whitneys(5)

Vector space of degree 7 and dimension 1 over Finite Field of size 2
Basis matrix:
[0 0 1 0 0 0 0]

In [273]:
[J for J in Partitions(5)]

[[5], [4, 1], [3, 2], [3, 1, 1], [2, 2, 1], [2, 1, 1, 1], [1, 1, 1, 1, 1]]

### Images of $\Omega^H_n(\text{pt})$ inside $\Omega^O_n(\text{pt})$ for various $H$

The following functions define the additional relations (on top of the Wu relations) for various tangential structures:

In [163]:
def so_relations(n):
    return (sw_monomial(Partition([1])) * sw_monomial(J) for J in Partitions(n - 1))

In [164]:
def pinplus_relations(n):
    return (sw_monomial(Partition([2])) * sw_monomial(J) for J in Partitions(n - 2))

In [165]:
def pinminus_relations(n):
    return (sw_polynomial([Partition([2]), Partition([1,1])]) * sw_monomial(J) for J in Partitions(n - 2))

In [166]:
def spin_relations(n):
    return itertools.chain(so_relations(n), pinplus_relations(n))

In [202]:
def image_dims(n):
    '''Returns the dimensions of (O, SO, Spin, Pin^+, Pin^-) bordism inside unoriented bordism.'''
    unoriented = stiefel_whitneys(n)
    oriented = unoriented.intersection(relations_to_subspace(so_relations(n), n))
    pinplus = unoriented.intersection(relations_to_subspace(pinplus_relations(n), n))
    pinminus = unoriented.intersection(relations_to_subspace(pinminus_relations(n), n))
    spin = oriented.intersection(pinplus)
    return ([x.dimension() for x in (unoriented, oriented, spin, pinplus, pinminus)])

The following table shows the dimensions of the images of various bordism groups in unoriented bordism:

In [238]:
table(rows=[('n', 'O', 'SO', 'Spin', 'Pin^+', 'Pin^-')] + [[n] + image_dims(n) for n in (0..22)], header_row=True)

n    O    SO   Spin   Pin^+   Pin^-
├────┼────┼────┼──────┼───────┼───────┤
  0    1    1    1      1       1
  1    0    0    0      0       0
  2    1    0    0      0       1
  3    0    0    0      0       0
  4    2    1    0      1       0
  5    1    1    0      0       0
  6    3    0    0      0       1
  7    1    0    0      0       0
  8    5    2    1      2       1
  9    3    2    0      0       0
  10   8    1    1      1       3
  11   5    1    0      1       1
  12   12   3    0      3       1
  13   9    4    0      1       0
  14   18   2    0      1       3
  15   14   3    0      0       1
  16   27   6    2      5       3
  17   23   8    0      1       0
  18   40   5    2      3       7
  19   35   7    0      2       3
  20   58   11   1      8       4
  21   54   15   0      4       1
  22   84   11   1      5       9

Here is an optimised version of the above just for $\text{pin}^-$ manifolds:

In [274]:
def pinminus_dim(n):
    rels = itertools.chain(wu_relations(n), pinminus_relations(n))
    return relations_to_matrix(rels, n).right_nullity()

In [275]:
pinminus_dim(26)

17

In [276]:
pinminus_dim(30)

23

In [277]:
pinminus_dim(34)

41

In [278]:
pinminus_dim(38)

56